In [1]:
# Some jupyter notebook magic to reload modules automaticaally when they change
# not neccessary for this specific notebook but useful in general
%load_ext autoreload
%autoreload 2

In [2]:
from definitions import ROOT_DIR
import os

# The PSDM specific input models can be imported from the pypsdm.models.input and
# pypsdm.models.result. The `GridWithResults` container is located in pypsdm.models.gwr
from pypsdm.models.gwr import GridWithResults


grid_path = os.path.join(ROOT_DIR, "tests", "resources", "simbench", "input")
result_path = os.path.join(ROOT_DIR, "tests", "resources", "simbench", "results")
# IO data models in general have a from_csv method to parse psdm files
gwr = GridWithResults.from_csv("simbench", grid_path, result_path)

In [3]:
from pypsdm.plots.grid import grid_plot

# Use the grid_plot method to visualize the grid model
# only works if the underlying node input files have associated coordinates
grid_plot(gwr.grid)

In [4]:
results = gwr.results
# The grid results are symmetrical to the input grid, so there is a result container
# for participants and for the raw grid.
raw_grid_res = results.raw_grid
participants_res = results.participants

In [5]:
# Results for each grid element and participant is a discrete event time series
# which basically means each recorded state of e.g. a node (consisting of current magnitude, and angle)
# is valid until the next recorded state.

nodes_res = raw_grid_res.nodes
# Reminder: you can also access the nodes result directly from the gwr e.g. gwr.nodes_res
nodes_res = gwr.nodes_res

In [6]:
# The underlying structure of the result data is a dictionary named entities
# the values of the dictionary are dependent on the result model
# e.g. the result dictionar NodesResult contains NodeResult objects
from pypsdm.models.result.grid.node import NodeResult


entities = nodes_res.entities

# You can access items as you would with a standard dictionary
uuid = list(entities.keys())[0]
node_res: NodeResult = nodes_res[uuid]  # type: ignore

# Similar to the input models the underlying data structure of each singular result object
# is a pandas DataFrame with a date time index
node_res.data.head()

,v_ang,v_mag
time,,
2016-01-01 00:00:00,-1.370031,1.032337
2016-01-01 03:00:00,-1.567882,1.031248
2016-01-01 04:00:00,-1.466684,1.031595
2016-01-01 05:00:00,-1.793983,1.029550
2016-01-01 06:00:00,-1.679715,1.030489


In [7]:
from datetime import datetime


# If you want to look at some time interval, you can slice the result modesl with date
# time objects
start = datetime(2016, 1, 1)
end = datetime(2016, 1, 1, 1)
sliced_node_res = node_res[start:end]
sliced_node_res.data

,v_ang,v_mag
time,,
2016-01-01 00:00:00,-1.370031,1.032337
2016-01-01 01:00:00,-1.370031,1.032337


In [8]:
from pypsdm.models.result.grid.node import NodesResult


# You can also filter the whole dictionary for a time interval
sliced_nodes_res = nodes_res.filter_for_time_interval(start, end)
sliced_nodes_res[uuid].data

,v_ang,v_mag
time,,
2016-01-01 00:00:00,-1.370031,1.032337
2016-01-01 01:00:00,-1.370031,1.032337


In [9]:
# For participants all things look mostly the same
# The type of results are mostly of type PQResult, containing active and reactive power
from pypsdm.models.result.power import PQResult


loads_res = participants_res.loads
load_res = loads_res[list(loads_res.entities.keys())[0]]
load_res.data.head()

,p,q
time,,
2016-01-01 00:00:00,0.100361,0.033545
2016-01-01 01:00:00,0.079933,0.025773
2016-01-01 02:00:00,0.132742,0.042507
2016-01-01 03:00:00,0.064642,0.021374
2016-01-01 04:00:00,0.083497,0.028292


In [10]:
# There are calculation utilities implemented where it makes sense
load_res2 = loads_res[list(loads_res.entities.keys())[1]]
load_sum = load_res + load_res2
load_sum.data.head()

,p,q
time,,
2016-01-01 00:00:00,0.108260,0.036790
2016-01-01 01:00:00,0.086937,0.028719
2016-01-01 02:00:00,0.141324,0.046164
2016-01-01 03:00:00,0.076531,0.026630
2016-01-01 04:00:00,0.095099,0.033547


In [11]:
# You can also calculate the sum for all participants
total_loads_res = loads_res.sum()
total_loads_res.data.head()

,p,q
time,,
2016-01-01 00:00:00,10.000451,3.294543
2016-01-01 01:00:00,8.531443,2.888455
2016-01-01 02:00:00,9.787666,3.346276
2016-01-01 03:00:00,9.121015,3.169117
2016-01-01 04:00:00,11.186874,3.714538


In [12]:
# If you care about a specific attribute of all entities in a result dict you can assemble
# a data frame with all time series of all entities in the dict
loads_res_p = loads_res.p.head()
loads_res_p

,0014e974-d490-4a1c-96e2-23d0e523b104,02a26927-da09-410f-8452-903177ef645b,05bdcb09-d1b4-4a30-b06a-b26d543d6929,07ff3b74-bda4-4a30-9584-3daa033068bb,095d4836-9075-49c1-bb2b-447db22fcb22,0acbb234-af80-44b3-8c5a-cff35e3bd7ac,0df30eab-565f-4184-8404-c33f80a3d1a6,0e64c305-f846-4bd4-8898-fbd833469bde,0ec9fd06-f5db-423c-96f1-03ddb88b9491,1199d1f7-22ae-47ad-91b4-e89c9f8fd16f,...,eecfdcb5-b327-48e3-a525-832d0209e7b8,f06c94f9-5ee5-4e60-bc20-3a2f62699f4d,f0b13ceb-da8a-4767-a441-d36980c15b72,f2447245-8a45-4980-8498-cf471d886365,f251d2a7-f131-4614-b906-418fa5750765,f2704836-9871-4f0d-8e62-433f2c95ede4,f2840dba-9c0c-48e9-b5b0-a25499ae51dd,f4d3b235-5403-4188-88a1-94b80dfa53b8,f6204e43-1cfd-4cd8-91b2-3846a980154f,ff389df3-f90d-41ef-b967-d92c0a51e918
time,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,0.100361,0.007899,0.034845,0.055080,0.110637,0.042421,0.122813,0.005346,0.002746,0.036699,...,0.061879,0.009083,0.038949,0.092180,0.008062,0.059149,0.028435,0.044687,0.124652,0.087379
2016-01-01 01:00:00,0.079933,0.007004,0.024737,0.027209,0.112610,0.023127,0.119688,0.002618,0.002270,0.030516,...,0.046179,0.008223,0.037156,0.098700,0.003986,0.037158,0.024806,0.029256,0.116439,0.068687
2016-01-01 02:00:00,0.132742,0.008582,0.061820,0.098530,0.147473,0.029389,0.120650,0.002033,0.013430,0.038521,...,0.041041,0.008213,0.046452,0.146509,0.003548,0.048451,0.057203,0.018476,0.138171,0.080424
2016-01-01 03:00:00,0.064642,0.011889,0.024960,0.032690,0.118495,0.030611,0.127332,0.004005,0.004047,0.022036,...,0.037481,0.008235,0.026474,0.099523,0.005493,0.022540,0.029163,0.014499,0.110900,0.072728
2016-01-01 04:00:00,0.083497,0.011602,0.084249,0.122456,0.132410,0.038047,0.128587,0.004869,0.015812,0.040018,...,0.044955,0.008369,0.086329,0.175877,0.006207,0.052840,0.062008,0.017805,0.133994,0.076716


In [13]:
# It is generally easier to interpret the data with the entities ids.
# This works for all types of results
load_uuid_id_map = gwr.loads.id.to_dict()
loads_res_p.rename(columns=load_uuid_id_map)

,MV3.101 Load 35,MV3.101 Load 128,MV3.101 Load 32,MV3.101 Load 4,MV3.101 Load 102,MV3.101 Load 9,MV3.101 Load 127,MV3.101 Load 39,MV3.101 Load 30,MV3.101 Load 57,...,MV3.101 Load 124,MV3.101 MV Load 4,MV3.101 Load 41,MV3.101 Load 56,MV3.101 Load 22,MV3.101 Load 75,MV3.101 Load 83,MV3.101 Load 126,MV3.101 Load 132,MV3.101 Load 99
time,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,0.100361,0.007899,0.034845,0.055080,0.110637,0.042421,0.122813,0.005346,0.002746,0.036699,...,0.061879,0.009083,0.038949,0.092180,0.008062,0.059149,0.028435,0.044687,0.124652,0.087379
2016-01-01 01:00:00,0.079933,0.007004,0.024737,0.027209,0.112610,0.023127,0.119688,0.002618,0.002270,0.030516,...,0.046179,0.008223,0.037156,0.098700,0.003986,0.037158,0.024806,0.029256,0.116439,0.068687
2016-01-01 02:00:00,0.132742,0.008582,0.061820,0.098530,0.147473,0.029389,0.120650,0.002033,0.013430,0.038521,...,0.041041,0.008213,0.046452,0.146509,0.003548,0.048451,0.057203,0.018476,0.138171,0.080424
2016-01-01 03:00:00,0.064642,0.011889,0.024960,0.032690,0.118495,0.030611,0.127332,0.004005,0.004047,0.022036,...,0.037481,0.008235,0.026474,0.099523,0.005493,0.022540,0.029163,0.014499,0.110900,0.072728
2016-01-01 04:00:00,0.083497,0.011602,0.084249,0.122456,0.132410,0.038047,0.128587,0.004869,0.015812,0.040018,...,0.044955,0.008369,0.086329,0.175877,0.006207,0.052840,0.062008,0.017805,0.133994,0.076716


## Nodal Results

Often times we care about nodal results, so the net load and generation at individual 
nodes. Therefore we include a special data model `EnhancedNodesResult` which is 
calculated by summing up the results of all connected participants at the indivdual nodes.

In [14]:
# There is a method to calculate all of them
nodal_results = gwr.build_enhanced_nodes_result()

target_uuid = nodal_results.uuids()[3]
nodal_results[target_uuid].data.head()

,v_ang,v_mag,p,q
time,,,,
2016-01-01 00:00:00,-1.568923,1.026038,0.217446,0.097896
2016-01-01 01:00:00,-1.568923,1.026038,0.192289,0.088259
2016-01-01 02:00:00,-1.568923,1.026038,0.182469,0.084294
2016-01-01 03:00:00,-1.818521,1.023885,0.192557,0.092193
2016-01-01 04:00:00,-1.673757,1.024844,0.239855,0.109596


In [15]:
# If you care about the individual systems at each node, you can filter down the
# result container to the systems connected to a specified node

nodal_gwr = gwr.nodal_result(target_uuid)
nodal_gwr.loads.p

,a0330517-9705-4d0a-bcaf-71f203cd6187
time,
2016-01-01 00:00:00,0.172376
2016-01-01 01:00:00,0.161575
2016-01-01 02:00:00,0.153038
2016-01-01 03:00:00,0.162732
2016-01-01 04:00:00,0.166878
...,...
2016-01-07 20:00:00,0.180501
2016-01-07 21:00:00,0.182086
2016-01-07 22:00:00,0.165844


In [16]:
target_uuid

'090d13e8-3cce-4793-816f-4c50f23f3f7f'

In [17]:
# Let's say you want to make sure that all the load is actually connected to the
# node that we have viltered for
load_uuids = nodal_gwr.loads.uuids()

# You can get a subset of the input data model via a list of uuids
# Note that the `node` attribute confirms that indeed the load is connected
# to the node we have filtered for
gwr.loads.subset(load_uuids).data

,cos_phi_rated,dsm,e_cons_annual,id,load_profile,node,operates_from,operates_until,operator,q_characteristics,s_rated
uuid,,,,,,,,,,,
a0330517-9705-4d0a-bcaf-71f203cd6187,0.93,False,0.0,MV3.101 Load 61,No load profile assigned,090d13e8-3cce-4793-816f-4c50f23f3f7f,NaN,NaN,NaN,"cosPhiFixed:{(0.0,0.93)}",668.0


For more details please check the implemented methods of all the data models 

Check out the `docs/nbs/plots.ipynb` notebook for some examples of the included plotting utilities  